# Project: Wrangle and Analyze Data
By: Christoph Lindstädt
## Table of Contents
<ul>
    <li><a href="#intro">Introduction</a></li>
    <li><a href="#questions">Posing Questions</a></li>
    <li><a href="#sources">Data Sources</a></li>
    <li><a href="#gathering">Data Gathering</a></li>
    <li><a href="#assessing">Data Assessing</a></li>
    <ul>
        <li><a href="#assessingsum">Assessing Summary</a></li>            
    </ul>    
    <li><a href="#cleaning">Data Cleaning</a></li>
    <li><a href="#analysis">Data Analysis</a></li>
    <ul>
        <li><a href="#uni">Univariate Exploration</a></li>
        <li><a href="#bi">Bivariate Exploration</a></li>
        <li><a href="#multi">Multivariate Exploration</a></li> 
    </ul> 
    <li><a href="#conclusion">Summary and Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

<a id='questions'></a>
## Posing Questions

The following analysis will take a look on the following questions:
><ol>
>    <li><p>Q1</p></li>     
>    <li><p>Q2</p></li>     
></ol>

<a id='sources'></a>
## Data Sources

>1. **Name:**</li>
><ul>   
>    <li><b>Source:</b> <a href =""></a></li>    
>    <li><b>Description:</b></li>
>    <li><b>Version:</b></li>
>    <li><b>Method of gathering:</b> Manual download</li>
></ul>


#### Import of the needed libraries:

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

<a id='gathering'></a>
## Data Gathering

<a id='assessing'></a>
## Data Assessing

<a id='assessingsum'></a>
### Assessing Summary

#### Quality
##### "name" table
- quality issue


#### Tidiness
##### "name" table
- tidiness issue

<a id='cleaning'></a>
## Data Cleaning

### Issue #1
> #### Define #1
> Text describing the issue

> #### Clean #1

In [4]:
# Code

> #### Test #1

In [5]:
# Code

<a id='analysis'></a>
## Data Analysis

<a id='uni'></a>
### Univariate Exploration

> Distribution, unusual points, transformations, changes in data - why?

<a id='bi'></a>
### Bivariate Exploration
> Relationships

<a id='multi'></a>
### Multivariate Exploration
> Relationships

<a id='conclusion'></a>
## Summary and Conclusions

**Discussed questions:**
><ol>
>    <li><p><b>Question</b></p></li>
> Answer 
>    <li><p><b>Question</b></p></li>
> Answer
></ol>

**Limitations due to the cleaning process in the used datasets:**

><ol>
    ><li><p>Describing limitations to the original dataset</li>
></ol>

**Summary**
><ol>
>  Summary and Conclusions
</ol>